In [2]:
import torch
from PIL import Image
import open_clip
import numpy as np

model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k', device="cpu", jit=False)
text = open_clip.tokenize(["MRI"])
print(f'text1={text.shape}')
text_features1 = model.encode_text(text)
print(f'text_features_ViT-B-32: {text_features1.shape}') #torch.Size([1, 512]): tensor([[ 1.3201e-01, -1.7446e-01, -3.3043e-01, -3.1494e-01, -3.9451e-02,

model, _, preprocess = open_clip.create_model_and_transforms('ViT-L-14', pretrained='openai', device="cpu", jit=False)
text = open_clip.tokenize(["MRI"])
print(f'text2={text.shape}')
text_features1 = model.encode_text(text)
print(f'text_features_ViT-L-14: {text_features1.shape}') #torch.Size([1, 768]): tensor([[-1.9208e-01,  2.1010e-02,  1.9233e-01, -1.8018e-01, -9.7014e-02,

model, _, preprocess = open_clip.create_model_and_transforms('coca_ViT-L-14', pretrained='mscoco_finetuned_laion2b_s13b_b90k', device="cpu", jit=False)
text = open_clip.tokenize(["MRI"])
print(f'text3={text.shape}')
text_features1 = model.encode_text(text)
print(f'text_features_ViT-B-32: {text_features1.shape}') #torch.Size([1, 768]): tensor([[ 1.8017e-02,  4.3765e-02,  7.0854e-03,  6.3915e-03,  1.1728e-02,

text1=torch.Size([1, 77])
text_features_ViT-B-32: torch.Size([1, 512])
text2=torch.Size([1, 77])
text_features_ViT-L-14: torch.Size([1, 768])
text3=torch.Size([1, 77])
text_features_ViT-B-32: torch.Size([1, 768])


In [1]:
# Load standard CLIP model, image, text on cpu
import torch
from PIL import Image
import open_clip
import numpy as np
from clip_onnx import clip_onnx, attention

model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k', device="cpu", jit=False)
image_path = '/mnt/eds_share/share/Spine2D/GlobusSrgMapData_crop_square/test/images/anon_d4730414.dcm_lateral_full_lumbar_thoracic.jpg'
# model.eval()
# batch first
image = preprocess(Image.open(image_path)).unsqueeze(0)  # [1, 3, 224, 224]
image_onnx = image.detach().cpu().numpy().astype(np.float32)

# batch first
# tokenizer = open_clip.get_tokenizer('ViT-B-32')
# text = tokenizer(["MRI", "Xray", "CT"])
text = open_clip.tokenize(["MRI", "Xray", "CT"]) #
text_onnx = text.detach().cpu().numpy().astype(np.int64)
print(f'text_onnx.shape: {text_onnx.shape}')

#Create CLIP-ONNX object to convert model to onnx
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=-1, keepdims=True)

visual_path = "clip_visual.onnx"
textual_path = "clip_textual.onnx"

onnx_model = clip_onnx(model, visual_path=visual_path, textual_path=textual_path)
onnx_model.convert2onnx(image, text, verbose=True)
# ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
onnx_model.start_sessions(providers=["CPUExecutionProvider"]) # cpu mode

#Use for standard CLIP API. Batch inference
image_features = onnx_model.encode_image(image_onnx)
# print(f'image_features: {image_features}')
text_features = onnx_model.encode_text(text_onnx)
print(f'text_features_onnx.shape: {text_features.shape}')
logits_per_image, logits_per_text = onnx_model(image_onnx, text_onnx)
text_probs = logits_per_image.softmax(dim=-1).detach().cpu().numpy()
# image_features /= np.linalg.norm(image_features, axis=-1, keepdims=True)
# text_features /= np.linalg.norm(text_features, axis=-1, keepdims=True)
# text_probs = softmax(100.0 * image_features @ text_features.T)
print("Label probs:", text_probs)  # prints: [[0.27080908 0.44203502 0.28715587]]

# with torch.no_grad(), torch.cuda.amp.autocast():
image_features = model.encode_image(image)
# print(f'image_features: {image_features}')
text_features1 = model.encode_text(text)
print(f'text_features1_openclip.shape: {text_features1.shape}')
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features1 /= text_features1.norm(dim=-1, keepdim=True)
text_probs = (100.0 * image_features @ text_features1.T).softmax(dim=-1)
print("Label probs:", text_probs)  # prints: tensor([[0.0197, 0.9333, 0.0471]])

text_onnx.shape: (3, 77)
[CLIP ONNX] Start convert visual model
[CLIP ONNX] Start check visual model
[CLIP ONNX] Start convert textual model


/opt/miniconda/envs/yz_openclip/lib/python3.10/site-packages/torch/onnx/symbolic_opset9.py:5856: UserWarning: Exporting aten::index operator of advanced indexing in opset 14 is achieved by combination of multiple ONNX operators, including Reshape, Transpose, Concat, and Gather. If indices include negative values, the exported graph will produce incorrect results.
  warnings.warn(


[CLIP ONNX] Start check textual model
[CLIP ONNX] Models converts successfully
text_features_onnx.shape: (3, 512)
Label probs: [[0.01968996 0.93325    0.04706011]]
test:test
text_features1_openclip.shape: torch.Size([3, 512])
Label probs: tensor([[0.0197, 0.9333, 0.0471]])


In [1]:
# Load standard OPENCLIP Coca model, image, text on cpu
import torch
from PIL import Image
import open_clip
import numpy as np
from clip_onnx import clip_onnx

model, _, preprocess = open_clip.create_model_and_transforms('coca_ViT-L-14', pretrained='mscoco_finetuned_laion2b_s13b_b90k', device="cpu", jit=False)
image_path = '/mnt/eds_share/share/Spine2D/GlobusSrgMapData_crop_square/test/images/anon_d4730414.dcm_lateral_full_lumbar_thoracic.jpg'
model.eval()
# batch first
image = preprocess(Image.open(image_path)).unsqueeze(0)  # [1, 3, 224, 224]
image_onnx = image.detach().cpu().numpy().astype(np.float32)

# batch first
# tokenizer = open_clip.get_tokenizer('ViT-B-32')
# text = tokenizer(["MRI", "Xray", "CT"])
text = open_clip.tokenize(["MRI", "Xray", "CT"]) #
text_onnx = text.detach().cpu().numpy().astype(np.int64)
print(f'text_onnx.shape: {text_onnx.shape}')

#Create CLIP-ONNX object to convert model to onnx
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=-1, keepdims=True)

visual_path = "clip_visual.onnx"
textual_path = "clip_textual.onnx"

# onnx_model = clip_onnx(model, visual_path=visual_path, textual_path=textual_path)
# onnx_model.convert2onnx(image, text, verbose=True)
# ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
# onnx_model.start_sessions(providers=["CPUExecutionProvider"]) # cpu mode

#Use for standard CLIP API. Batch inference
# image_features = onnx_model.encode_image(image_onnx)
# text_features = onnx_model.encode_text(text_onnx)
# print(f'text_features_onnx.shape: {text_features.shape}')
# logits_per_image, logits_per_text = onnx_model(image_onnx, text_onnx)
# text_probs = logits_per_image.softmax(dim=-1).detach().cpu().numpy()
# # image_features /= np.linalg.norm(image_features, axis=-1, keepdims=True)
# # text_features /= np.linalg.norm(text_features, axis=-1, keepdims=True)
# # text_probs = softmax(100.0 * image_features @ text_features.T)
# print("Label probs:", text_probs)  # prints: [[0.27080908 0.44203502 0.28715587]]

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    # print(f'image_features: {image_features}')
    text_features1 = model.encode_text(text)
    print(f'text_features1_openclip.shape: {text_features1.shape}')
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features1 /= text_features1.norm(dim=-1, keepdim=True)
    text_probs = (100.0 * image_features @ text_features1.T).softmax(dim=-1)
    print("Label probs:", text_probs)  # prints: tensor([[0.0197, 0.9333, 0.0471]])

TODO: _build_text_tower______________________
TODO: embed_dim = 768
TODO: text_cfg = CLIPTextCfg(context_length=76, vocab_size=49408, width=768, heads=12, layers=12, ls_init_value=None, hf_model_name=None, hf_tokenizer_name=None, hf_model_pretrained=True, proj='mlp', pooler_type='mean_pooler', embed_cls=True, pad_id=0, output_tokens=True)
TODO: quick_gelu = False
TODO: cast_dtype = None
text_onnx.shape: (3, 77)
text_features1_openclip.shape: torch.Size([3, 768])
Label probs: tensor([[1.5892e-01, 8.4095e-01, 1.3302e-04]])


In [4]:
tokenizer = open_clip.get_tokenizer('ViT-B-32')
# text = tokenizer(["MRI", "Xray", "CT"])
# tokenizer = AutoTokenizer.from_pretrained('roberta-base')
token = tokenizer.decode([ 49406, 24773, 49407,     0,     0,])
print(token)

AttributeError: 'function' object has no attribute 'decode'

In [6]:
# caption prediction
import open_clip
import torch
from PIL import Image
#('coca_ViT-B-32', 'laion2b_s13b_b90k')
# model, _, transform = open_clip.create_model_and_transforms(
#     model_name="coca_ViT-B-32",  #coca_ViT-L-14, 
#     pretrained="laion2b_s13b_b90k", #mscoco_finetuned_laion2B-s13B-b90k
# )
# /mnt/eds_share/Users/yilu.zhou/Development/log/open_clip/2023_07_28-13_55_44-model_coca_ViT-L-14-lr_1e-05-b_32-j_4-p_amp/checkpoints/epoch_1.pt
model, _, transform = open_clip.create_model_and_transforms(
    model_name="coca_ViT-L-14",
    pretrained="/mnt/eds_share/Users/yilu.zhou/Development/log/open_clip_GlobusSrgMapData_crop_square/2023_08_08-22_02_21-model_coca_ViT-L-14-lr_1e-06-b_32-j_4-p_amp/checkpoints/epoch_78.pt"
)


In [3]:
import torch
print(torch.__version__)

2.1.0.dev20230816+cu121


In [3]:
#classification for ONNX
import torch
from PIL import Image
import numpy as np
import open_clip

# /mnt/eds_share/Users/yilu.zhou/Development/log/open_clip/2023_07_28-13_55_44-model_coca_ViT-L-14-lr_1e-05-b_32-j_4-p_amp/checkpoints/epoch_3.pt
model, _, preprocess = open_clip.create_model_and_transforms(
    model_name="coca_ViT-L-14",
    pretrained="/mnt/eds_share/Users/yilu.zhou/Development/log/open_clip_GlobusSrgMapData_crop_square/2023_08_08-22_02_21-model_coca_ViT-L-14-lr_1e-06-b_32-j_4-p_amp/checkpoints/epoch_78.pt"
)
model = model.eval()
tokenizer = open_clip.get_tokenizer('coca_ViT-L-14')

image_path = '/mnt/eds_share/share/Spine2D/GlobusSrgMapData_crop_square/test/images/anon_d4730414.dcm_lateral_full_lumbar_thoracic.jpg'
im = Image.open(image_path).convert("RGB")
im = im.resize((224, 224), Image.Resampling.LANCZOS)
dummy_image = preprocess(im).unsqueeze(0)
sentences = ["thoracic", "lumbar", "thoracic lumbar"]
dummy_texts = tokenizer(sentences) # , "Xray"

torch.onnx.export(model, (dummy_image, dummy_texts), "clip_resnet.onnx", export_params=True,
  input_names=["IMAGE", "TEXT"],
  output_names=["LOGITS_PER_IMAGE", "LOGITS_PER_TEXT"],
  opset_version=14,
  dynamic_axes={
      "IMAGE": {
          0: "image_batch_size",
      },
      "TEXT": {
          0: "text_batch_size",
      },
      "LOGITS_PER_IMAGE": {
          0: "image_batch_size",
          1: "text_batch_size",
      },
      "LOGITS_PER_TEXT": {
          0: "text_batch_size",
          1: "image_batch_size",
      },
  }
)

# image_features = model.encode_image(dummy_image)
# text_features = model.encode_text(dummy_texts)

# image_features /= image_features.norm(dim=-1, keepdim=True)
# text_features /= text_features.norm(dim=-1, keepdim=True)

# text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
# text_probs = text_probs.cpu().tolist()[0]

# # Construct the dictionary
# class_predict_dict = dict(zip(sentences, text_probs))
# # Extract the key with the largest value
# class_predict = max(class_predict_dict, key=class_predict_dict.get)
# print(f"class_predict: {class_predict}")  # prints: [[1., 0., 0.]]

#https://github.com/openai/CLIP/pull/219

/opt/miniconda/envs/yz_openclip/lib/python3.10/site-packages/open_clip/coca_model.py:137: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  text = text[:, :-1] if embed_cls else text # make space for CLS token


RuntimeError: shape '[255, 36, 64]' is invalid for input of size 195840

In [6]:
import cv2
import torch
from albumentations import Resize, Compose
from albumentations.pytorch.transforms import  ToTensor
from albumentations.augmentations.transforms import Normalize

tokenizer = open_clip.get_tokenizer('coca_ViT-L-14')
sentences = ["thoracic", "lumbar", "thoracic lumbar"]
dummy_texts = tokenizer(sentences) 

image_path = '/mnt/eds_share/share/Spine2D/GlobusSrgMapData_crop_square/test/images/anon_d4730414.dcm_lateral_full_lumbar_thoracic.jpg'
im = Image.open(image_path).convert("RGB")
im = im.resize((224, 224), Image.Resampling.LANCZOS)
dummy_image = preprocess(im).unsqueeze(0)



ONNX_FILE_PATH = 'openclip.onnx'
# torch.onnx.export(model, (dummy_image, dummy_texts), ONNX_FILE_PATH, input_names=["IMAGE", "TEXT"],
#                   output_names=["LOGITS_PER_IMAGE", "LOGITS_PER_TEXT"], export_params=True)

torch.onnx.export(model, (dummy_image, dummy_texts), ONNX_FILE_PATH, input_names=["IMAGE", "TEXT"],
                  output_names=["LOGITS_PER_TEXT"], export_params=True)

RuntimeError: shape '[255, 36, 64]' is invalid for input of size 195840

In [9]:
# caption prediction
image_path = '/mnt/eds_share/share/Spine2D/GlobusSrgMapData_crop_square/test/images/anon_d4730414.dcm_lateral_full_lumbar_thoracic.jpg'
im = Image.open(image_path).convert("RGB")
im = im.resize((224, 224), Image.Resampling.LANCZOS)
im = transform(im).unsqueeze(0)

with torch.no_grad(), torch.cuda.amp.autocast():
    generated = model.generate(im)

print(open_clip.decode(generated[0]).split("<end_of_text>")[0].replace("<start_of_text>", ""))

anteroposterior thoracic lumbar 


In [4]:
# Load standard CLIP model, image, text on cpu
import open_clip
from PIL import Image
import numpy as np
import torch
# onnx cannot work with cuda
model, _, preprocess = open_clip.create_model_and_transforms(
    model_name="coca_ViT-L-14",
    pretrained="/mnt/eds_share/Users/yilu.zhou/Development/log/open_clip_GlobusSrgMapData_crop_square/2023_08_08-22_02_21-model_coca_ViT-L-14-lr_1e-06-b_32-j_4-p_amp/checkpoints/epoch_78.pt"
)
# model, preprocess = clip.load("ViT-B/32", device="cpu", jit=False)

# batch first
image_path = '/mnt/eds_share/share/Spine2D/GlobusSrgMapData_crop_square/test/images/anon_d4730414.dcm_lateral_full_lumbar_thoracic.jpg'
im = Image.open(image_path).convert("RGB")
im = im.resize((224, 224), Image.Resampling.LANCZOS)
image = preprocess(im).unsqueeze(0).cpu() # [1, 3, 224, 224]
# image = preprocess(Image.open("CLIP.png")).unsqueeze(0).cpu() # [1, 3, 224, 224]
image_onnx = image.detach().cpu().numpy().astype(np.float32)

# batch first
tokenizer = open_clip.get_tokenizer('coca_ViT-L-14')
sentences = ["thoracic", "lumbar", "thoracic lumbar"]
text = tokenizer(sentences).cpu()
# text = clip.tokenize(["a diagram", "a dog", "a cat"]).cpu() # [3, 77]
text_onnx = text.detach().cpu().numpy().astype(np.int32)

In [13]:
# https://github.com/pytorch/pytorch/issues/97262
import onnxscript

# Assuming you use opset17
from onnxscript.onnx_opset import opset13 as op

custom_opset = onnxscript.values.Opset(domain="torch.onnx", version=1)


@onnxscript.script(custom_opset)
def ScaledDotProductAttention(
    query,
    key,
    value,
    dropout_p,
):
    # Swap the last two axes of key
    key_shape = op.Shape(key)
    key_last_dim = key_shape[-1:]
    key_second_last_dim = key_shape[-2:-1]
    key_first_dims = key_shape[:-2]
    # Contract the dimensions that are not the last two so we can transpose
    # with a static permutation.
    key_squeezed_shape = op.Concat(
        op.Constant(value_ints=[-1]), key_second_last_dim, key_last_dim, axis=0
    )
    key_squeezed = op.Reshape(key, key_squeezed_shape)
    key_squeezed_transposed = op.Transpose(key_squeezed, perm=[0, 2, 1])
    key_transposed_shape = op.Concat(key_first_dims, key_last_dim, key_second_last_dim, axis=0)
    key_transposed = op.Reshape(key_squeezed_transposed, key_transposed_shape)

    embedding_size = op.CastLike(op.Shape(query)[-1], query)
    scale = op.Div(1.0, op.Sqrt(embedding_size))

    # https://github.com/pytorch/pytorch/blob/12da0c70378b5be9135c6fda62a9863bce4a4818/aten/src/ATen/native/transformers/attention.cpp#L653
    # Scale q, k before matmul for stability see https://tinyurl.com/sudb9s96 for math
    query_scaled = op.Mul(query, op.Sqrt(scale))
    key_transposed_scaled = op.Mul(key_transposed, op.Sqrt(scale))
    attn_weight = op.Softmax(
        op.MatMul(query_scaled, key_transposed_scaled),
        axis=-1,
    )
    attn_weight, _ = op.Dropout(attn_weight, dropout_p)
    return op.MatMul(attn_weight, value)


def custom_scaled_dot_product_attention(g, query, key, value, attn_mask, dropout, is_causal, scale=None):
    return g.onnxscript_op(ScaledDotProductAttention, query, key, value, dropout).setType(value.type())


torch.onnx.register_custom_op_symbolic(
    symbolic_name="aten::scaled_dot_product_attention",
    symbolic_fn=custom_scaled_dot_product_attention,
    opset_version=18,
)

In [5]:
# Create CLIP-ONNX object to convert model to onnx
from clip_onnx import clip_onnx

visual_path = "clip_visual.onnx"
textual_path = "clip_textual.onnx"

onnx_model = clip_onnx(model, visual_path=visual_path, textual_path=textual_path)
onnx_model.convert2onnx(image, text, verbose=True)
# ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
onnx_model.start_sessions(providers=["CPUExecutionProvider"]) # cpu mode

#Use for standard CLIP API. Batch inference
image_features = onnx_model.encode_image(image_onnx)
text_features = onnx_model.encode_text(text_onnx)

logits_per_image, logits_per_text = onnx_model(image_onnx, text_onnx)
probs = logits_per_image.softmax(dim=-1).detach().cpu().numpy()

print("Label probs:", probs)  # prints: [[0.9927937  0.00421067 0.00299571]]

# https://github.com/pytorch/pytorch/issues/96944

# /opt/miniconda/envs/yz_openclip/lib/python3.10/site-packages/clip_onnx/utils.py

[CLIP ONNX] Start convert visual model
[CLIP ONNX] Start check visual model
[CLIP ONNX] Start convert textual model


AttributeError: 'CoCa' object has no attribute 'transformer'